In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
%matplotlib inline
import arcpy

In [2]:
arcpy.env.workspace = './data/cyklo_alko.gdb/'
arcpy.env.overwriteOutput = True

In [2]:
data = pd.read_csv(r'../nehody-chodci/scratch/nehody_all.csv')

In [3]:
data.columns

Index(['X', 'Y', 'id', 'obec', 'datum', 'komunikace_druh', 'komunikace_cislo',
       'zavineni', 'ridic_kategorie', 'ridic_ovlivneni', 'vinik_alkohol',
       'ridic_stav', 'obeti_usmrceno', 'obeti_lehce', 'obeti_tezce',
       'vozidel_celkem', 'vinik_vozidlo_znacka', 'vinik_vozidlo_po_nehode',
       'skoda', 'unik_hmot', 'vinik_vozidlo_druh', 'vinik_vozidlo_rok_vyroby',
       'vinik_vozidlo_vlastnik', 'vinik_skoda_na_vozidle', 'vyprosteni',
       'nehoda_charakter', 'srazka_vozidel_druh', 'hlavni_priciny',
       'povrch_stav', 'meteo', 'rozhled', 'nehoda_poloha', 'prednost_v_jizde',
       'smerove_podminky', 'druh_krizujici_komunikace', 'smer_jizdy',
       'nehoda_druh', 'pevna_prekazka_druh', 'povrch_druh', 'komunikace_stav',
       'viditelnost', 'komunikace_deleni', 'rizeni_provozu', 'objekty',
       'misto_nehody', 'smyk'],
      dtype='object')

In [5]:
alko = data[(data.vinik_vozidlo_druh == 'jízdní kolo') & (data.vinik_alkohol.str.contains('alkoholu'))]

In [6]:
alko.datum = pd.to_datetime(alko.datum)

C:\Anaconda3\lib\site-packages\pandas\core\generic.py:2177: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [7]:
#nehody v case
serie = pd.Series(np.ones(len(alko)), index=alko['datum'])
serie.resample('A', how='sum')#.plot('bar')

datum
2007-12-31    465
2008-12-31    439
2009-12-31    528
2010-12-31    534
2011-12-31    644
2012-12-31    731
2013-12-31    731
2014-12-31    794
2015-12-31    181
Freq: A-DEC, dtype: float64

In [8]:
alko14 = alko[(alko.datum >= '01-01-2014') & (alko.datum < '01-01-2015')]

In [9]:
alko14.vinik_alkohol.value_counts()

ano, obsah alkoholu v&nbsp;krvi 1,5‰ a více        533
ano, obsah alkoholu v&nbsp;krvi od 1,0‰ do 1,5‰    133
ano, obsah alkoholu v&nbsp;krvi od 0,5‰ do 0,8‰     45
ano, obsah alkoholu v&nbsp;krvi od 0,8‰ do 1,0‰     33
ano - obsah alkoholu v krvi 1‰ a více               30
ano, obsah alkoholu v&nbsp;krvi do 0,99‰            18
pod vlivem drog a alkoholu                           2
dtype: int64

In [38]:
alko.viditelnost.value_counts()

ve dne, viditelnost nezhoršená vlivem po    2995
v noci - s veřejným osvětlením,viditelno    1087
v noci - bez veřejného osvětlení,viditel     620
ve dne, zhoršená viditelnost (svítání,so     136
v noci - s veřejným osvětlením, zhoršená      91
ve dne, zhoršená viditelnost vlivem pově      67
v noci - bez veřejného osvětlení, vidite      51
dtype: int64

In [3]:
opravy = pd.read_excel('data/opravy_prodlouzeni.xlsx')

In [5]:
viditelnost = []

for row in opravy.iterrows():
    if row[1]['viditelnost'] not in viditelnost:
        viditelnost.append(row[1]['viditelnost'])
        
viditelnost = viditelnost[0:len(viditelnost)-2]


fc = 'cyklo_alko_tbl'

cursor = arcpy.UpdateCursor(fc)
for row in cursor:
    old = row.getValue("viditelnost")
    for val in viditelnost:
        if (val.startswith(old)):
            row.setValue("viditelnost", val)
            cursor.updateRow(row)
            continue

In [37]:
alko.to_csv('data/cyklo_alko.csv', encoding='utf-8', index=False)